# Benchmarking with Our Test Split

This notebook details how you can benchmark with our test split using any function.

## Installs and Imports

In [14]:
# !curl -s https://packagecloud.io/install/repositories/github/git-lfs/script.rpm.sh | sudo bash
# !sudo yum install git-lfs -y
# !git lfs install

In [2]:
%%capture
%pip install torch==1.9.0+cpu torchvision==0.10.0+cpu torchaudio==0.9.0 -f https://download.pytorch.org/whl/torch_stable.html
# %pip install torch==1.9.0+cu111 torchvision==0.10.0+cu111 torchaudio==0.9.0 -f https://download.pytorch.org/whl/torch_stable.html
%pip install datasets==1.11.0
%pip install transformers==4.9.1
%pip install jiwer
%pip install ipywidgets
%pip install pythainlp==2.3.1
%pip install pydub
%pip install SpeechRecognition
%pip install azure-cognitiveservices-speech

In [1]:
from functools import partial
import pandas as pd
import numpy as np
from datasets import (
    load_dataset, 
    load_from_disk,
    load_metric,)
from datasets.filesystems import S3FileSystem
from transformers import (
    Wav2Vec2CTCTokenizer, 
    Wav2Vec2FeatureExtractor,
    Wav2Vec2Processor,
    Wav2Vec2ForCTC,
    TrainingArguments,
    Trainer,
)
import torch
import torchaudio
import re
import json
from pythainlp.tokenize import word_tokenize, syllable_tokenize
from tqdm.auto import tqdm

## Download Dataset

We download the whole Common Voice 7.0 even though we will use only our test split for benchmarking. We also copy `train_cleaned.tsv`, `dev_cleaned.tsv`, and `test_cleaned.tsv` for our cleaned and resampled splits.

In [2]:
# !wget https://voice-prod-bundler-ee1969a6ce8178826482b88e843c335139bd3fb4.s3.amazonaws.com/cv-corpus-7.0-2021-07-21/cv-corpus-7.0-2021-07-21-th.tar.gz
# !tar -xvf cv-corpus-7.0-2021-07-21-th.tar.gz --no-same-owner
# !mv cv-corpus-7.0-2021-07-21-th ../data
# !cp train_cleaned.tsv dev_cleaned.tsv test_cleaned.tsv ../data/cv-corpus-7.0-2021-07-21-th/th

## Load Test Set

In [3]:
test_dataset = load_dataset("../scripts/th_common_voice_70.py", "th", split="test")
test_dataset

0 examples [00:00, ? examples/s]

0 examples [00:00, ? examples/s]

0 examples [00:00, ? examples/s]

Dataset common_voice downloaded and prepared to /home/ec2-user/.cache/huggingface/datasets/common_voice/th/7.0.0/14bf435a174687b310ed94f56abf0198f6cc7efb5a5d945c22c83113eab67701. Subsequent calls will reuse this data.


Dataset({
    features: ['path', 'sentence'],
    num_rows: 2502
})

## Load Metrics

We use WER with words tokenized by PyThaiNLP 2.3.1 and CER.

In [4]:
from pythainlp.tokenize import word_tokenize

wer_metric = load_metric("wer")
cer_metric = load_metric("cer")

Downloading:   0%|          | 0.00/1.95k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.91k [00:00<?, ?B/s]

## Preprocess

In [5]:
def speech_file_to_array_fn(batch, 
                            text_col="sentence", 
                            fname_col="path",
                            resampling_to=16000):
    speech_array, sampling_rate = torchaudio.load(batch[fname_col])
    resampler=torchaudio.transforms.Resample(sampling_rate, resampling_to)
    batch["speech"] = resampler(speech_array)[0].numpy()
    batch["sampling_rate"] = resampling_to
    batch["target_text"] = batch[text_col]
    return batch

test_dataset = test_dataset.map(speech_file_to_array_fn)

  0%|          | 0/2502 [00:00<?, ?ex/s]

## Evaluate

### Convert to `.wav` for Some APIs

We need to convert the `mp3` files to `wav`. Install `ffmpeg and ffprobe`:

In [ ]:
# sudo su -
# mkdir -v -p /usr/local/bin/ffmpeg
# cd /usr/local/bin/ffmpeg
# wget https://johnvansickle.com/ffmpeg/releases/ffmpeg-release-i686-static.tar.xz
# tar -v -xf ffmpeg-release-i686-static.tar.xz --strip-components=1
# rm -v -f ffmpeg-release-i686-static.tar.xz
# ln -snf /usr/local/bin/ffmpeg/ffmpeg /usr/bin/ffmpeg
# ln -snf /usr/local/bin/ffmpeg/ffprobe /usr/bin/ffpropbe
# exit

In [8]:
from pydub import AudioSegment

def mp3_to_wav(example):
    example['path_wav'] = (example['path'][:-3] + 'wav').replace('clips','clips_wav')
    sound = AudioSegment.from_mp3(example['path'])
    sound.export(example['path_wav'], format="wav")
    return example

In [9]:
test_dataset = test_dataset.map(mp3_to_wav)
test_dataset

  0%|          | 0/2502 [00:00<?, ?ex/s]

Dataset({
    features: ['path', 'sentence', 'speech', 'sampling_rate', 'target_text', 'path_wav'],
    num_rows: 2502
})

### [Google Web Speech API](https://w3c.github.io/speech-api/speechapi.html)

In [118]:
import speech_recognition as sr

r = sr.Recognizer()

def evaluate_example(fname):
    with sr.AudioFile(fname) as source:
        audio_text = r.listen(source)
    text = r.recognize_google(audio_text, language = "th-TH")
    return text
        
evaluate_example(test_dataset[0]['path_wav'])

'และเขาก็สัมผัสดีบุก'

In [ ]:
ds = []
for i in tqdm(test_dataset):
    try:
        pred = evalute_example(i['path_wav']).replace(' ','')
    except:
        pred = ''
    d = {'sentence': i['sentence'].replace(' ',''), 
         'pred_sentence': pred}
    d['sentence_tok'] = ' '.join(word_tokenize(d['sentence']))
    d['pred_sentence_tok'] = ' '.join(word_tokenize(d['pred_sentence']))
    ds.append(d)

  0%|          | 0/2502 [00:00<?, ?it/s]

In [64]:
# result_df = pd.DataFrame(ds)
# result_df.to_csv('artifacts/result_google.csv',index=False)

result_df = pd.read_csv('artifacts/result_google.csv').fillna('')
result_df

,sentence,pred_sentence,sentence_tok,pred_sentence_tok
0,และเขาก็สัมผัสดีบุก,และเขาก็สัมผัสดีบุก,และ เขา ก็ สัมผัส ดีบุก,และ เขา ก็ สัมผัส ดีบุก
1,คุณสามารถรับทราบเมื่อข้อความนี้ถูกอ่านแล้ว,คุณสามารถรับทราบข้อความนี้ถูกอ่านแล้ว,คุณ สามารถ รับทราบ เมื่อ ข้อความ นี้ ถูก อ่าน ...,คุณ สามารถ รับทราบ ข้อความ นี้ ถูก อ่าน แล้ว
2,คืนนี้ฉันต้องทำให้ได้เธอพูดกับตัวเองฉันต้องทำใ...,คืนนี้ฉันต้องทำให้ได้เธอพูดกับตัวเองฉันต้องทำใ...,คืนนี้ ฉัน ต้อง ทำ ให้ได้ เธอ พูด กับ ตัวเอง ฉ...,คืนนี้ ฉัน ต้อง ทำ ให้ได้ เธอ พูด กับ ตัวเอง ฉ...
3,การทำเช่นนี้จะทำให้แผ่นถ่ายภาพเกิดแสงฟลูออเรสเ...,การทำเช่นนี้จะทำให้แผ่นถ่ายภาพเกิดแสงตัวartists,การ ทำ เช่นนี้ จะ ทำให้ แผ่น ถ่ายภาพ เกิด แสง ...,การ ทำ เช่นนี้ จะ ทำให้ แผ่น ถ่ายภาพ เกิด แสง ...
4,ผู้ปกครองของฉันไม่สนใจความเห็นนี้อย่างละเอียดอ...,ผู้ปกครองของฉันไม่สนใจความเห็นนี้อย่างละเอียดอ...,ผู้ปกครอง ของ ฉัน ไม่ สนใจ ความเห็น นี้ อย่าง ...,ผู้ปกครอง ของ ฉัน ไม่ สนใจ ความเห็น นี้ อย่าง ...
...,...,...,...,...
2497,ใครสนใจไปเรียนได้ฟรีวันละชั่วโมงสัปดาห์เดียวก็จบ,ใครสนใจจะเรียนได้ฟรีวันละชั่วโมงสัปดาห์เดียวก็จบ,ใคร สนใจ ไป เรียน ได้ ฟรี วัน ละ ชั่วโมง สัปดา...,ใคร สนใจ จะ เรียน ได้ ฟรี วัน ละ ชั่วโมง สัปดา...
2498,หรือในกรณีตั้งครรภ์เมื่ออายุเกิน,หรือในกรณีตั้งครรภ์เมื่ออายุเกิน,หรือ ใน กรณี ตั้งครรภ์ เมื่อ อายุ เกิน,หรือ ใน กรณี ตั้งครรภ์ เมื่อ อายุ เกิน
2499,ก่อนสเต็ปนั้น,ก่อนสเต็ปนั้น,ก่อน สเต็ป นั้น,ก่อน สเต็ป นั้น
2500,ถ้าจะแบนมันต้องชัดว่าคำพูดจะนำไปสู่เหตุการณ์พว...,ถ้าจะแบนมันต้องชัดว่าคำพูดจะนำไปสู่เหตุการณ์พว...,ถ้า จะ แบน มัน ต้อง ชัด ว่า คำพูด จะ นำไปสู่ เ...,ถ้า จะ แบน มัน ต้อง ชัด ว่า คำพูด จะ นำไปสู่ เ...


In [65]:
#wer
wer_metric.compute(predictions=result_df.pred_sentence_tok,references=result_df.sentence_tok)

0.1371123407540857

In [66]:
#cer
cer_metric.compute(predictions=result_df.pred_sentence,references=result_df.sentence)

0.07357340720221607

### [Microsoft Bing Speech API](https://azure.microsoft.com/en-us/services/cognitive-services/speech/)

In [ ]:
import azure.cognitiveservices.speech as speechsdk

API_KEY = 'YOUR_API_KEY'
REGION = 'YOUR_REGION'
speech_config = speechsdk.SpeechConfig(subscription=API_KEY, 
                                       region=REGION)
speech_config.speech_recognition_language = "th-TH"

def evaluate_example(fname):
    audio_input = speechsdk.AudioConfig(filename=fname)
    speech_recognizer = speechsdk.SpeechRecognizer(speech_config=speech_config, audio_config=audio_input)
    result = speech_recognizer.recognize_once_async().get()
    return result.text

evaluate_example(test_dataset[0]['path_wav'])

In [ ]:
ds = []
for i in tqdm(test_dataset):
    try:
        pred = evaluate_example(i['path_wav']).replace(' ','')
    except:
        pred = ''
    d = {'sentence': i['sentence'].replace(' ',''), 
         'pred_sentence': pred}
    d['sentence_tok'] = ' '.join(word_tokenize(d['sentence']))
    d['pred_sentence_tok'] = ' '.join(word_tokenize(d['pred_sentence']))
    ds.append(d)

In [11]:
# result_df = pd.DataFrame(ds)
# result_df.to_csv('artifacts/result_bing.csv',index=False)

result_df = pd.read_csv('artifacts/result_bing.csv').fillna('')
result_df

,sentence,pred_sentence,sentence_tok,pred_sentence_tok
0,และเขาก็สัมผัสดีบุก,และเขาก็สัมผัสดีบุ๊ค,และ เขา ก็ สัมผัส ดีบุก,และ เขา ก็ สัมผัส ดี บุ๊ค
1,คุณสามารถรับทราบเมื่อข้อความนี้ถูกอ่านแล้ว,คุณสามารถรับทราบเมื่อข้อความนี้ถูกอ่านแล้ว,คุณ สามารถ รับทราบ เมื่อ ข้อความ นี้ ถูก อ่าน ...,คุณ สามารถ รับทราบ เมื่อ ข้อความ นี้ ถูก อ่าน ...
2,คืนนี้ฉันต้องทำให้ได้เธอพูดกับตัวเองฉันต้องทำใ...,คืนนี้ฉันต้องทำให้ได้เธอพูดกับตัวเองฉันต้องทำใ...,คืนนี้ ฉัน ต้อง ทำ ให้ได้ เธอ พูด กับ ตัวเอง ฉ...,คืนนี้ ฉัน ต้อง ทำ ให้ได้ เธอ พูด กับ ตัวเอง ฉ...
3,การทำเช่นนี้จะทำให้แผ่นถ่ายภาพเกิดแสงฟลูออเรสเ...,การทำเช่นนี้จะทำให้แผ่นถ่ายภาพเกิดแสงตัวริมเส้น,การ ทำ เช่นนี้ จะ ทำให้ แผ่น ถ่ายภาพ เกิด แสง ...,การ ทำ เช่นนี้ จะ ทำให้ แผ่น ถ่ายภาพ เกิด แสง ...
4,ผู้ปกครองของฉันไม่สนใจความเห็นนี้อย่างละเอียดอ...,ผู้ปกครองของฉันไม่สนใจความเห็นนี้อย่างละเอียดอ...,ผู้ปกครอง ของ ฉัน ไม่ สนใจ ความเห็น นี้ อย่าง ...,ผู้ปกครอง ของ ฉัน ไม่ สนใจ ความเห็น นี้ อย่าง ...
...,...,...,...,...
2497,ใครสนใจไปเรียนได้ฟรีวันละชั่วโมงสัปดาห์เดียวก็จบ,ใครสนใจไปเรียนได้ฟรีวันละชั่วโมงสัปดาห์เดียวก็จบ,ใคร สนใจ ไป เรียน ได้ ฟรี วัน ละ ชั่วโมง สัปดา...,ใคร สนใจ ไป เรียน ได้ ฟรี วัน ละ ชั่วโมง สัปดา...
2498,หรือในกรณีตั้งครรภ์เมื่ออายุเกิน,หรือในกรณีตั้งครรภ์เมื่ออายุเกิน,หรือ ใน กรณี ตั้งครรภ์ เมื่อ อายุ เกิน,หรือ ใน กรณี ตั้งครรภ์ เมื่อ อายุ เกิน
2499,ก่อนสเต็ปนั้น,ก่อนสเต็ปนั้น,ก่อน สเต็ป นั้น,ก่อน สเต็ป นั้น
2500,ถ้าจะแบนมันต้องชัดว่าคำพูดจะนำไปสู่เหตุการณ์พว...,ถ้าจะแบนมันต้องชัดว่าคำพูดจะนำไปสู่เหตุการณ์พว...,ถ้า จะ แบน มัน ต้อง ชัด ว่า คำพูด จะ นำไปสู่ เ...,ถ้า จะ แบน มัน ต้อง ชัด ว่า คำพูด จะ นำไปสู่ เ...


In [12]:
#wer
wer_metric.compute(predictions=result_df.pred_sentence_tok,references=result_df.sentence_tok)

0.12578818684853943

In [13]:
#cer
cer_metric.compute(predictions=result_df.pred_sentence,references=result_df.sentence)

0.050166204986149585

## [AWS Transcribe](https://aws.amazon.com/transcribe/)

In [ ]:
import time
import boto3

s3 = boto3.resource("s3")
s3_bucket = s3.Bucket("cv-test-unique")
prefix = "th/clips_wav"
clips = [f.key.split('/')[-1] for f in s3_bucket.objects.filter(Prefix=prefix).all()]
clips = [i for i in clips if i[-3:]=='wav']
len(clips)

In [ ]:
ds = []

for i in tqdm(range(len(clips)//bs+1)):
    sub_clips = clips[i*bs:(i+1)*bs]
    for job_name in sub_clips:
        job_uri = f's3://cv-test-unique/{prefix}/{job_name}'
        transcribe.start_transcription_job(
            TranscriptionJobName=job_name,
            Media={'MediaFileUri': job_uri},
            MediaFormat='wav',
            LanguageCode='th-TH'
        )

    while True:
        dones = []
        for job_name in sub_clips:
            status = transcribe.get_transcription_job(TranscriptionJobName=job_name)
            dones.append(status['TranscriptionJob']['TranscriptionJobStatus'] in ['COMPLETED', 'FAILED'])
        if np.mean(dones)==1:
            print('Batch done')
            break
        print(f"{sum(dones)} out of {len(dones)} done for batch {i}")
        time.sleep(5)


    for job_name in sub_clips:
        status = transcribe.get_transcription_job(TranscriptionJobName=job_name)
        try:
            data = pd.read_json(status['TranscriptionJob']['Transcript']['TranscriptFileUri'])
            pred_text = data['results'][1][0]['transcript']
        except:
            pred_text = ''
        ds.append((job_name, ''.join(pred_text.split())))

    df = pd.DataFrame(ds)
    df.columns = ['fname','pred_sentence']
    df.to_csv(f'result_aws_{i}.csv',index=False)

In [ ]:
# ground_truth = pd.DataFrame(test_dataset)
# ground_truth['fname'] = ground_truth.path.map(lambda x: x.split('/')[-1])
# df['fname'] = df['fname'].map(lambda x: x.replace('.wav','.mp3'))
# result_df = ground_truth.merge(df,on='fname')
# result_df = result_df.drop(['fname','path'],1)
# result_df['pred_sentence'] = result_df['pred_sentence'].map(lambda x: x.replace('.','').replace(' ',''))
# result_df['sentence'] = result_df['sentence'].map(lambda x: x.replace(' ',''))
# result_df['sentence_tok'] = result_df.sentence.map(lambda x: ' '.join(word_tokenize(x)))
# result_df['pred_sentence_tok'] = result_df.pred_sentence.map(lambda x: ' '.join(word_tokenize(x)))
# result_df.to_csv('result_aws.csv',index=False)

In [11]:
result_df = pd.read_csv('artifacts/result_aws.csv').fillna('')

In [12]:
#wer
wer_metric.compute(predictions=result_df.pred_sentence_tok,references=result_df.sentence_tok)

0.21863338051730793

In [13]:
#cer
cer_metric.compute(predictions=result_df.pred_sentence,references=result_df.sentence)

0.07077562326869806

### [NECTEC AI for Thai Partii API](https://aiforthai.in.th/aiplatform/#/speechtotext)

We benchmark with the Partii API using instructions listed in [AI for Thai webiste](https://aiforthai.in.th/aiplatform/#/speechtotext).

In [119]:
import requests
 
url = "https://api.aiforthai.in.th/partii-webapi"
 
fname = test_dataset[0]['path_wav']
files = {'wavfile': (fname, open(fname, 'rb'), 'audio/wav')}

API_KEY = 'YOUR_API_KEY'
 
headers = {
    'Apikey': API_KEY,
    'Cache-Control': "no-cache",
    'Connection': "keep-alive",
    }
 
param = {"outputlevel":"--uttlevel","outputformat":"--txt"}
 
response = requests.request("POST", url, headers=headers, files=files, data=param)
 
print("Result = " + response.text)

Result = {"status":"False","message":"No asr result","inputfilename":"","outputfilename":""}


In [27]:
# #prediction
# df = pd.read_csv('artifacts/result_partii.tsv',sep='\t') #result called by vistec-ai/nectec-POSTer by @tann9949
# df.columns = ['fname','text']
# df['fname'] = df['fname'].map(lambda x: x+'.mp3')
# df['pred_sentence'] = df.text.map(lambda x: ''.join(x.replace(' ','').split()))
# df['pred_sentence_tok'] = df.pred_sentence.map(lambda x: ' '.join(word_tokenize(x)))
# df = df[['fname','pred_sentence','pred_sentence_tok']]

# #ground truth
# ground_truth = pd.DataFrame({'path':test_dataset['path'], 'sentence':test_dataset['sentence']})
# ground_truth['fname'] = ground_truth.path.map(lambda x: x.split('/')[-1])

# #merge
# result_df = ground_truth.merge(df,on='fname')
# result_df = result_df.drop(['fname','path'],1)
# result_df['sentence'] = result_df['sentence'].map(lambda x: x.replace(' ',''))
# result_df['sentence_tok'] = result_df.sentence.map(lambda x: ' '.join(word_tokenize(x)))
# result_df.to_csv('result_partii.csv',index=False)

In [25]:
#wer
wer_metric.compute(predictions=result_df.pred_sentence_tok,references=result_df.sentence_tok)

0.2010588680196647

In [26]:
#cer
cer_metric.compute(predictions=result_df.pred_sentence,references=result_df.sentence)

0.0955102705784922

### `wav2vec2` Models from HuggingFace

In [19]:
processor = Wav2Vec2Processor.from_pretrained("airesearch/wav2vec2-large-xlsr-53-th")
model = Wav2Vec2ForCTC.from_pretrained("airesearch/wav2vec2-large-xlsr-53-th")

def evaluate(batch):
    device = "cuda" if torch.cuda.is_available() else "cpu"
    inputs = processor(batch["speech"], sampling_rate=16_000, return_tensors="pt", padding=True)

    with torch.no_grad():
        logits = model(inputs.input_values.to(device),).logits

    pred_ids = torch.argmax(logits, dim=-1)
    batch["pred_sentence"] = processor.batch_decode(pred_ids)
    return batch

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [25]:
#takes quite a long time on CPU
result = test_dataset.map(evaluate, batched=True, batch_size=8)
result_df = pd.DataFrame({'sentence':result['sentence'], 'pred_sentence_tok': result['pred_sentence']})
result_df['sentence_tok'] = result_df.sentence.map(lambda x: ' '.join(word_tokenize(x)))
result_df['pred_sentence'] = result_df.pred_sentence_tok.map(lambda x: ''.join(x.split()))
result_df

In [16]:
#load our results
result_df = pd.read_csv('artifacts/result_cv70.csv')

In [17]:
#wer
wer_metric.compute(predictions=result_df.pred_sentence_tok,references=result_df.sentence_tok)

0.13634023935143483

In [18]:
#cer
cer_metric.compute(predictions=result_df.pred_sentence,references=result_df.sentence)

0.028130193905817176